In [69]:
%pip install pyodbc

import pyodbc
import pandas as pd

server = 'LAPTOP-USMIAD3A\SQLEXPRESS'


#Connect to existing database
# connection = pyodbc.connect(
#     'Driver={SQL Server};'
#     'Server=LAPTOP-USMIAD3A\SQLEXPRESS;'
#     'Database=AdventureWorksDW2017;'
#     'Trusted_Connection=yes;'
# )

#Connect To Server
connection = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=LAPTOP-USMIAD3A\SQLEXPRESS;'
)

print("Connected")

Note: you may need to restart the kernel to use updated packages.
Connected


In [70]:
def sql(query):
    try:
        cursor = connection.cursor()
        cursor.execute(query)
        connection.commit()
        print("Done")
    except Exception as e:
        print(f"An error occurred: {e}")

In [71]:
# #Using cursor.execute
# cursor.execute("SELECT TOP 10 FirstName FROM DimEmployee")
# results = cursor.fetchall()
# for row in results:
#     print(row)

# #Using pd.read_sql
# sql_query = "SELECT TOP 10 * FROM DimEmployee"
# df = pd.read_sql(sql_query, connection)
# df

database="businessERP"
sql("CREATE DATABASE businessERP")

Done


In [72]:
connection = pyodbc.connect(
    'Driver={SQL Server};'
    'Server=LAPTOP-USMIAD3A\SQLEXPRESS;'
    'Database=businessERP;'
    'Trusted_Connection=yes;'
)

print("Done")

Done


In [73]:
def pdsql(sql_query):
    try:
        df = pd.read_sql(sql_query, connection)
        return df
    
    except Exception as e:
        print(f"An error occurred: {e}")

In [74]:
sql("CREATE TABLE company(companyID int, companyName varchar(200), createdDate DATETIME)")

Done


In [75]:
sql("INSERT INTO company (companyID, companyName, createdDate) VALUES (1, 'Livepath', convert(datetime,'28-03-23 00:00:00 AM',5))")

Done


In [76]:
sql_query = "SELECT TOP 2 * FROM company"
df = pd.read_sql(sql_query, connection)
df

,companyID,companyName,createdDate
0,1,Livepath,2023-03-28


In [77]:
sql("CREATE TABLE employees(employeeID int, first_name VARCHAR(200), last_name VARCHAR(200), email VARCHAR(200), gender VARCHAR(200), related_company INT)")

Done


In [78]:
df = pd.read_csv(r'C:\Users\jorge\OneDrive\Escritorio\Backup OneDrive\DatasetSQLs\employees.csv')

df.head()

,id,first_name,last_name,email,gender,related_company
0,1,Bernie,Mogford,bmogford0@ow.ly,Male,1
1,2,Mignonne,Baumler,mbaumler1@google.com.au,Female,1
2,3,Kain,Chaney,kchaney2@usda.gov,Male,1
3,4,Templeton,Woodyer,twoodyer3@amazon.co.jp,Non-binary,1
4,5,Putnem,Singleton,psingleton4@google.com.br,Male,1


In [79]:
cursor = connection.cursor()

for row in df.itertuples():
    cursor.execute("INSERT INTO businessERP.dbo.employees (employeeID, first_name, last_name, email, gender, related_company) VALUES (?, ?, ?, ?, ?, ?)", 
                   row.id, row.first_name, row.last_name, row.email, row.gender, row.related_company)
    
connection.commit()

In [80]:
pdsql("SELECT TOP 10 * FROM employees")

,employeeID,first_name,last_name,email,gender,related_company
0,1,Bernie,Mogford,bmogford0@ow.ly,Male,1
1,2,Mignonne,Baumler,mbaumler1@google.com.au,Female,1
2,3,Kain,Chaney,kchaney2@usda.gov,Male,1
3,4,Templeton,Woodyer,twoodyer3@amazon.co.jp,Non-binary,1
4,5,Putnem,Singleton,psingleton4@google.com.br,Male,1
5,6,Darb,Gloster,dgloster5@mysql.com,Female,1
6,7,Hube,Lazare,hlazare6@npr.org,Male,1
7,8,Ina,Izsak,iizsak7@indiegogo.com,Female,1
8,9,Stinky,Helks,shelks8@is.gd,Male,1
9,10,Gannie,Worley,gworley9@usnews.com,Male,1


In [81]:
pdsql("SELECT table_name FROM INFORMATION_SCHEMA.TABLES")

,table_name
0,company
1,employees


In [82]:
df = pd.read_csv(r'C:\Users\jorge\OneDrive\Escritorio\Backup OneDrive\DatasetSQLs\inventoryProducts.csv')

df.head()

,id,product_name,product_category,related_company,created_date
0,1,"Acetaminophen, Doxylamine succinate , Phenylep...",indomethacin,1,1/10/2022
1,2,Erythromycin Stearate,Nateglinide,1,9/20/2022
2,3,"Eucalyptol, menthol, methyl salicylate, thymol",PredniSONE Intensol,1,4/3/2022
3,4,Isosorbide Mononitrate,Chlorhexidine Gluconate,1,7/30/2023
4,5,Aluminum Chlorohydrate,Zinc Oxide and Calamine,1,5/11/2022


In [83]:
sql("CREATE TABLE products (product_id INT, product_name VARCHAR(500), product_category VARCHAR(200), units_purchased FLOAT DEFAULT 0.0, units_sold FLOAT DEFAULT 0.0, available_units AS (units_purchased - units_sold), related_company INT, created_date DATETIME)")

Done


In [84]:
pdsql("SELECT * FROM products")

,product_id,product_name,product_category,units_purchased,units_sold,available_units,related_company,created_date


In [85]:
for row in df.itertuples():
    cursor.execute("INSERT INTO businessERP.dbo.products (product_id, product_name, product_category, related_company, created_date) VALUES (?, ?, ?, ?, ?)", 
                   row.id, row.product_name, row.product_category, row.related_company, row.created_date)
    
connection.commit()

In [86]:
pdsql("SELECT TOP 10 * FROM products")

,product_id,product_name,product_category,units_purchased,units_sold,available_units,related_company,created_date
0,1,"Acetaminophen, Doxylamine succinate , Phenylep...",indomethacin,0.0,0.0,0.0,1,2022-01-10
1,2,Erythromycin Stearate,Nateglinide,0.0,0.0,0.0,1,2022-09-20
2,3,"Eucalyptol, menthol, methyl salicylate, thymol",PredniSONE Intensol,0.0,0.0,0.0,1,2022-04-03
3,4,Isosorbide Mononitrate,Chlorhexidine Gluconate,0.0,0.0,0.0,1,2023-07-30
4,5,Aluminum Chlorohydrate,Zinc Oxide and Calamine,0.0,0.0,0.0,1,2022-05-11
5,6,Baclofen,Ibuprofen,0.0,0.0,0.0,1,2023-08-10
6,7,Hydroxyzine Pamoate,Octinoxate,0.0,0.0,0.0,1,2022-06-23
7,8,"Ibuprofen, Pseudoephedrine Hydrochloride","Acetaminophen, Caffeine, Pyrilamine maleate",0.0,0.0,0.0,1,2022-04-23
8,9,Sincalide,"Avobenzone, Octinoxate, and Octisalate",0.0,0.0,0.0,1,2023-03-03
9,10,HYALURONIC ACID,Magesium Citrate,0.0,0.0,0.0,1,2022-01-17


In [87]:
pdsql("SELECT table_name FROM INFORMATION_SCHEMA.TABLES")

,table_name
0,company
1,employees
2,products


In [88]:
df = pd.read_csv(r"C:\Users\jorge\OneDrive\Escritorio\Backup OneDrive\DatasetSQLs\inventoryMovements.csv")

df

,id,product_id,units,movement_date,purchaseOrSale
0,1,36,16,2/3/2023,Purchase
1,2,15,89,29/1/2023,Sale
2,3,43,50,22/9/2022,Purchase
3,4,8,55,25/9/2022,Sale
4,5,20,16,26/3/2023,Purchase
...,...,...,...,...,...
995,996,104,59,19/5/2023,Purchase
996,997,18,87,2/1/2023,Purchase
997,998,76,2,19/12/2022,Purchase
998,999,115,6,9/3/2023,Sale


In [89]:
sql("CREATE TABLE inventory_movements(movement_id INT, product_id INT, units FLOAT, movement_date DATETIME, purchaseOrSale VARCHAR(50))")

Done


In [90]:
pdsql("SELECT * FROM inventory_movements")

,movement_id,product_id,units,movement_date,purchaseOrSale


In [91]:
from datetime import datetime

for row in df.itertuples():

    formatted_date = datetime.strptime(str(row.movement_date), '%d/%m/%Y').strftime('%Y-%m-%d')

    cursor.execute("INSERT INTO businessERP.dbo.inventory_movements(movement_id, product_id, units, movement_date, purchaseOrSale) VALUES (?, ?, ?, ?, ?)", 
                   row.id, row.product_id, row.units, formatted_date, row.purchaseOrSale)
    
connection.commit()

In [92]:
pdsql("SELECT TOP 12 * FROM inventory_movements")

,movement_id,product_id,units,movement_date,purchaseOrSale
0,1,36,16.0,2023-03-02,Purchase
1,2,15,89.0,2023-01-29,Sale
2,3,43,50.0,2022-09-22,Purchase
3,4,8,55.0,2022-09-25,Sale
4,5,20,16.0,2023-03-26,Purchase
5,6,47,28.0,2023-07-27,Purchase
6,7,128,67.0,2023-09-16,Sale
7,8,79,52.0,2023-03-06,Sale
8,9,147,80.0,2022-12-09,Purchase
9,10,144,60.0,2023-01-22,Sale


In [93]:
#Delete database
#sql("USE master; ALTER DATABASE businessERP SET SINGLE_USER WITH ROLLBACK IMMEDIATE; DROP DATABASE businessERP;")

#sql("DROP TABLE inventory_movements")

In [94]:
#connection.close()